# LSTM Test - Kaza - PCA

In [ ]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader

In [ ]:
DATASET_PATH = config.DATA_KAZA
jsonl_file = DATASET_PATH / "sampleKaza.jsonl"
if jsonl_file.exists():
    df = pd.read_json(jsonl_file, lines=True)
    if "traffic_sequence" in df.columns:
        df = pd.DataFrame(df["traffic_sequence"].tolist())
else:
    df = pd.read_parquet(DATASET_PATH / "sampleKaza.parquet")
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [ ]:
PCA_COMPONENTS = 5
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=PCA_COMPONENTS)
predictor = TrafficPredictor('lstm', dataset.sensors, X_STEP, Y_STEP, pca_model=dataset.pca if True else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/lstm_kaza_pca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
preds[:5]